In [23]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import seaborn as sns
import re
from sklearn.impute import KNNImputer
import warnings
warnings.filterwarnings('ignore') 

sns.set_theme(style = "white", context = "talk",font_scale = 0.5)
pd.set_option("display.max_columns", None)
pd.set_option('display.width', 1000)

In [24]:
sc = pd.read_json('scout_car.json', lines=True)

In [25]:
sc_1 = sc.copy()

In [26]:
sc.shape

(15919, 54)

In [5]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,kW,hp,Type,Previous Owners,Next Inspection,Inspection new,Warranty,Full Service,Non-smoking Vehicle,null,Make,Model,Offer Number,First Registration,Body Color,Paint Type,Body Color Original,Upholstery,Body,Nr. of Doors,Nr. of Seats,Model Code,Gearing Type,Displacement,Cylinders,Weight,Drive chain,Fuel,Consumption,CO2 Emission,Emission Class,\nComfort & Convenience\n,\nEntertainment & Media\n,\nExtras\n,\nSafety & Security\n,description,Emission Label,Gears,Country version,Electricity consumption,Last Service Date,Other Fuel Types,Availability,Last Timing Belt Service Date,Available from
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n2\n,"[\n06/2021\n, \n99 g CO2/km (comb)\n]","[\nYes\n, \nEuro 6\n]","[\n, \n, \n4 (Green)\n]","[\n, \n]","[\n, \n]",[],\nAudi\n,"[\n, A1, \n]",[\nLR-062483\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nMythosschwarz\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n5\n],[\n5\n],[\n0588/BDF\n],"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,220 kg\n]",[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Armrest, Automatic climate ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Catalytic Converter, Voice Cont...","[ABS, Central door lock, Daytime running light...","[\n, Sicherheit:, , Deaktivierung für Beifahr...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,NaN,141 kW,"[, Used, , Gasoline]",NaN,NaN,NaN,NaN,NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2017, \n]","[\n, Red, \n]",NaN,NaN,"[\nCloth, Grey\n]","[\n, Sedans, \n]",[\n3\n],[\n4\n],[\n0588/BCY\n],"[\n, Automatic, \n]","[\n1,798 cc\n]",[\n4\n],"[\n1,255 kg\n]",[\nfront\n],"[\n, Gasoline, \n]","[[5.6 l/100 km (comb)], [7.1 l/100 km (city)],...",[\n129 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Automatic climate control, ...","[Bluetooth, Hands-free equipment, On-board com...","[Alloy wheels, Sport seats, Sport suspension, ...","[ABS, Central door lock, Central door lock wit...",[\nLangstreckenfahrzeug daher die hohe Kilomet...,[\n4 (Green)\n],[\n7\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,NaN,85 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]",NaN,NaN,[],\nAudi\n,"[\n, A1, \n]",[\nAM-95365\n],"[\n, 2016, \n]","[\n, Black, \n]",[\nMetallic\n],[\nmythosschwarz metallic\n],"[\nCloth, Black\n]","[\n, Sedans, \n]",[\n4\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,598 cc\n]",NaN,NaN,[\nfront\n],"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.4 l/100 km (city)],...",[\n99 g CO2/km (comb)\n],[\nEuro 6\n],"[Air conditioning, Cruise control, Electrical ...","[MP3, On-board computer]","[Alloy wheels, Voice Control]","[ABS, Central door lock, Daytime running light...","[\n, Fahrzeug-Nummer: AM-95365, , Ehem. UPE 2...",[\n4 (Green)\n],NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,NaN,66 kW,"[, Used, , Diesel (Particulate Filter)]",\n1\n,NaN,NaN,NaN,"[\n, \n, \n99 g CO2/km (comb)\n]","[\n, \n, \nEuro 6\n]",[],\nAudi\n,"[\n, A1, \n]",NaN,"[\n, 2016, \n]","[\n, Brown, \n]",[\nMetallic\n],NaN,NaN,"[\n, Sedans, \n]",[\n3\n],[\n4\n],NaN,"[\n, Automatic, \n]","[\n1,422 cc\n]",[\n3\n],"[\n1,195 kg\n]",NaN,"[\n, Diesel (Particulate Filter), \n]","[[3.8 l/100 km (comb)], [4.3 l/100 km (city)],...",[

In [27]:
sc.columns = sc.columns.str.strip("\n")
sc.columns = sc.columns.str.replace(' ', '_')
sc.columns = sc.columns.str.replace('-', '_')

In [28]:
sc = sc.drop(columns = ["kW", "null","Make","Model","Offer_Number","First_Registration","Electricity_consumption","Last_Service_Date","Other_Fuel_Types","Availability","Last_Timing_Belt_Service_Date","Available_from"] )

In [29]:
cl_parantez = [ 'Type', 'Previous_Owners','Next_Inspection','Inspection_new','Warranty', 'Full_Service','Non_smoking_Vehicle', 'Body_Color', 'Paint_Type', 'Body_Color_Original', 'Upholstery', 'Body', 'Nr._of_Doors', 'Nr._of_Seats', 'Model_Code' ,'Gearing_Type', 'Displacement', 'Cylinders', 'Weight', 'Drive_chain', 'Fuel', 'Consumption', 'CO2_Emission', 'Emission_Class', 'Comfort_&_Convenience', 'Entertainment_&_Media' ,'Extras','Safety_&_Security','description','Emission_Label','Gears' ,'Country_version']

In [30]:
cl_parantez


['Type',
 'Previous_Owners',
 'Next_Inspection',
 'Inspection_new',
 'Warranty',
 'Full_Service',
 'Non_smoking_Vehicle',
 'Body_Color',
 'Paint_Type',
 'Body_Color_Original',
 'Upholstery',
 'Body',
 'Nr._of_Doors',
 'Nr._of_Seats',
 'Model_Code',
 'Gearing_Type',
 'Displacement',
 'Cylinders',
 'Weight',
 'Drive_chain',
 'Fuel',
 'Consumption',
 'CO2_Emission',
 'Emission_Class',
 'Comfort_&_Convenience',
 'Entertainment_&_Media',
 'Extras',
 'Safety_&_Security',
 'description',
 'Emission_Label',
 'Gears',
 'Country_version']

In [31]:
def try_join(l):
    try:
        return ','.join(map(str, l))
    except TypeError:
        return np.nan

In [34]:
#? sc["Consumption"] = [try_join(l) for l in sc["Consumption"]]

In [32]:
for i in cl_parantez:
     sc[i] = [try_join(l) for l in sc[i]]
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,",Used,,Diesel (Particulate Filter)","\n,2,\n","\n06/2021\n,\n99 g CO2/km (comb)\n","\nYes\n,\nEuro 6\n","\n,\n,\n4 (Green)\n","\n,\n","\n,\n","\n,Black,\n",\nMetallic\n,\nMythosschwarz\n,"\nCloth, Black\n","\n,Sedans,\n",\n5\n,\n5\n,\n0588/BDF\n,"\n,Automatic,\n","\n1,422 cc\n",\n3\n,"\n1,220 kg\n",\nfront\n,"\n,Diesel (Particulate Filter),\n","['3.8 l/100 km (comb)'],['4.3 l/100 km (city)'...",\n99 g CO2/km (comb)\n,\nEuro 6\n,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...","\n,Sicherheit:, ,Deaktivierung für Beifahrer-A...",NaN,NaN,NaN
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,141 kW,",Used,,Gasoline",NaN,NaN,NaN,NaN,NaN,NaN,"\n,Red,\n",NaN,NaN,"\nCloth, Grey\n","\n,Sedans,\n",\n3\n,\n4\n,\n0588/BCY\n,"\n,Automatic,\n","\n1,798 cc\n",\n4\n,"\n1,255 kg\n",\nfront\n,"\n,Gasoline,\n","['5.6 l/100 km (comb)'],['7.1 l/100 km (city)'...",\n129 g CO2/km (comb)\n,\nEuro 6\n,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic...","ABS,Central door lock,Central door lock with r...",\nLangstreckenfahrzeug daher die hohe Kilomete...,\n4 (Green)\n,\n7\n,NaN
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,",Used,,Diesel (Particulate Filter)","\n,1,\n",NaN,NaN,"\n,\n,\n99 g CO2/km (comb)\n",NaN,NaN,"\n,Black,\n",\nMetallic\n,\nmythosschwarz metallic\n,"\nCloth, Black\n","\n,Sedans,\n",\n4\n,\n4\n,NaN,"\n,Automatic,\n","\n1,598 cc\n",NaN,NaN,\nfront\n,"\n,Diesel (Particulate Filter),\n","['3.8 l/100 km (comb)'],['4.4 l/100 km (city)'...",\n99 g CO2/km (comb)\n,\nEuro 6\n,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,D...","\n,Fahrzeug-Nummer: AM-95365, ,Ehem. UPE 24.64...",\n4 (Green)\n,NaN,NaN
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,66 kW,",Used,,Diesel (Particulate Filter)","\n,1,\n",NaN,NaN,NaN,"\n,\n,\n99 g CO2/km (comb)\n","\n,\n,\nEuro 6\n","\n,Brown,\n",\nMetallic\n,NaN,NaN,"\n,Sedans,\n",\n3\n,\n4\n,NaN,"\n,Automatic,\n","\n1,422 cc\n",\n3\n,"\n1,195 kg\n",NaN,"\n,Diesel (Particulate Filter),\n","['3.8 l/100 km (comb)'],['4.3 l/100 km (city)'...",\n99 g CO2/km (comb)\n,\nEuro 6\n,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport seats,Voice Control","ABS,Alarm system,Central door lock with remote...","\nAudi A1: ,- 1e eigenaar ,- Perfecte staat: s...",NaN,\n6\n,NaN
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,66 kW,",Used,,Diesel (Particulate Filter)","\n,1,\n",NaN,"\nYes\n,\n109 g CO2/km (comb)\n","\n,\n,\nEuro 6\n","\n,\n,\n4 (Green)\n","\n,\n","\n,Black,\n",\nMetallic\n,\nMythosschwarz Metallic\n,"\nCloth, Black\n",

In [15]:
sc.isnull().sum()

url                          0
make_model                   0
short_description           46
body_type                   60
price                        0
vat                       4513
km                           0
registration                 0
prev_owner                6828
hp                           0
Type                         2
Previous_Owners           6640
Next_Inspection          12384
Inspection_new           11987
Warranty                  5420
Full_Service              7704
Non_smoking_Vehicle       8742
Body_Color                 597
Paint_Type                5772
Body_Color_Original       3759
Upholstery                3720
Body                        60
Nr._of_Doors               212
Nr._of_Seats               977
Model_Code               10941
Gearing_Type                 0
Displacement               496
Cylinders                 5680
Weight                    6974
Drive_chain               6858
Fuel                         0
Consumption               1906
CO2_Emis

In [111]:
#sc['Type'] = [','.join(map(str, l)) for l in sc['Type']]
#sc['Next_Inspection'] = [','.join(map(str, l)) for l in sc['Next_Inspection']]
sc['Inspection_new'] = [','.join(map(str, l)) for l in sc['Inspection_new']]
sc['Non_smoking_Vehicle'] = [','.join(map(str, l)) for l in sc['Non_smoking_Vehicle']]
sc['Model'] = [','.join(map(str, l)) for l in sc['Model']]
sc['Offer_Number'] = [','.join(map(str, l)) for l in sc['Offer_Number']]
sc['First_Registration'] = [','.join(map(str, l)) for l in sc['First_Registration']]
sc['Body_Color'] = [','.join(map(str, l)) for l in sc['Body_Color']]
sc['Body_Color_Original'] = [','.join(map(str, l)) for l in sc['Body_Color_Original']]
sc['Paint_Type'] = [','.join(map(str, l)) for l in sc['Paint_Type']]
sc['Upholstery'] = [','.join(map(str, l)) for l in sc['Upholstery']]
sc['Body'] = [','.join(map(str, l)) for l in sc['Body']]
sc['Nr._of_Doors'] = [','.join(map(str, l)) for l in sc['Nr._of_Doors']]
sc['Nr._of_Seats'] = [','.join(map(str, l)) for l in sc['Nr._of_Seats']]
sc['Model_Code'] = [','.join(map(str, l)) for l in sc['Model_Code']]
sc['Gearing_Type'] = [','.join(map(str, l)) for l in sc['Gearing_Type']]
sc['Displacement'] = [','.join(map(str, l)) for l in sc['Displacement']]
sc['Cylinders'] = [','.join(map(str, l)) for l in sc['Cylinders']]
sc['Weight'] = [','.join(map(str, l)) for l in sc['Weight']]
sc['Drive_chain'] = [','.join(map(str, l)) for l in sc['Drive_chain']]

TypeError: 'float' object is not iterable

In [89]:
for i in sc.columns:
    if type(i) == int:
        continue
        sc[i] = [','.join(map(str, l)) for l in sc[i]]
        print(i)
        
    


In [ ]:
Next_Inspection	Inspection_new	Warranty	Full_Service	Non_smoking_Vehicle	Body_Color	Paint_Type	Body_Color_Original	Upholstery	Body	Nr._of_Doors	Nr._of_Seats	Model_Code	Gearing_Type	Displacement	Cylinders	Weight	Drive_chain	Fuel
CO2_Emission	Emission_Class description	Emission_Label	Gears	Country_version

In [ ]:
#### Filling with '0'

In [33]:
sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].fillna(0)

In [ ]:
#### Removing '\n' 

In [34]:
sc[[ "Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].replace("\n","", regex = True)

In [ ]:
#### Removing ',' 

In [44]:
sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]] = sc[["Previous_Owners","Next_Inspection", "Inspection_new","Warranty","Full_Service","Non_smoking_Vehicle","Body_Color","Paint_Type", "Body_Color_Original","Upholstery","Body","Nr._of_Doors","Nr._of_Seats","Model_Code","Gearing_Type","Displacement","Cylinders", "Weight","Drive_chain","Fuel","CO2_Emission","Emission_Class","description","Emission_Label","Gears","Country_version"]].replace(",","", regex = True)

In [44]:
sc.isnull().sum()

url                          0
make_model                   0
short_description           46
body_type                   60
price                        0
vat                       4513
km                           0
registration                 0
prev_owner                6828
hp                           0
Type                         2
Previous_Owners           6870
Next_Inspection          12384
Inspection_new           11987
Warranty                  5420
Full_Service              7704
Non-smoking_Vehicle       8742
Body_Color                 597
Paint_Type                5772
Body_Color_Original       3759
Upholstery                3720
Body                        60
Nr._of_Doors               212
Nr._of_Seats               977
Model_Code               10941
Gearing_Type             15919
Displacement               496
Cylinders                 5680
Weight                    6974
Drive_chain               6858
Fuel                         0
Consumption               1906
CO2_Emis

In [46]:
missing_values = [" ", "[]", "Nan", None, "None", "NaN", ""]

In [ ]:
missing = sc.Full_Service.isin([missing_values]

In [ ]:
sc.mask(missing, np.NaN)

In [45]:
sc.head()

,url,make_model,short_description,body_type,price,vat,km,registration,prev_owner,hp,Type,Previous_Owners,Next_Inspection,Inspection_new,Warranty,Full_Service,Non_smoking_Vehicle,Body_Color,Paint_Type,Body_Color_Original,Upholstery,Body,Nr._of_Doors,Nr._of_Seats,Model_Code,Gearing_Type,Displacement,Cylinders,Weight,Drive_chain,Fuel,Consumption,CO2_Emission,Emission_Class,Comfort_&_Convenience,Entertainment_&_Media,Extras,Safety_&_Security,description,Emission_Label,Gears,Country_version
0,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-tronic Xenon Navi Klima,Sedans,15770,VAT deductible,"56,013 km",01/2016,2 previous owners,66 kW,",Used,,Diesel (Particulate Filter)",2,06/202199 g CO2/km (comb),YesEuro 6,4 (Green),,,Black,Metallic,Mythosschwarz,Cloth Black,Sedans,5,5,0588/BDF,Automatic,1422 cc,3,1220 kg,front,Diesel (Particulate Filter),"['3.8 l/100 km (comb)'],['4.3 l/100 km (city)'...",99 g CO2/km (comb),Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Catalytic Converter,Voice Control","ABS,Central door lock,Daytime running lights,D...",Sicherheit: Deaktivierung für Beifahrer-Airbag...,0,0,0
1,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.8 TFSI sport,Sedans,14500,Price negotiable,"80,000 km",03/2017,None,141 kW,",Used,,Gasoline",0,0,0,0,0,0,Red,0,0,Cloth Grey,Sedans,3,4,0588/BCY,Automatic,1798 cc,4,1255 kg,front,Gasoline,"['5.6 l/100 km (comb)'],['7.1 l/100 km (city)'...",129 g CO2/km (comb),Euro 6,"Air conditioning,Automatic climate control,Hil...","Bluetooth,Hands-free equipment,On-board comput...","Alloy wheels,Sport seats,Sport suspension,Voic...","ABS,Central door lock,Central door lock with r...",Langstreckenfahrzeug daher die hohe Kilometerl...,4 (Green),7,0
2,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.6 TDI S tronic Einparkhilfe plus+m...,Sedans,14640,VAT deductible,"83,450 km",02/2016,1 previous owner,85 kW,",Used,,Diesel (Particulate Filter)",1,0,0,99 g CO2/km (comb),0,0,Black,Metallic,mythosschwarz metallic,Cloth Black,Sedans,4,4,0,Automatic,1598 cc,0,0,front,Diesel (Particulate Filter),"['3.8 l/100 km (comb)'],['4.4 l/100 km (city)'...",99 g CO2/km (comb),Euro 6,"Air conditioning,Cruise control,Electrical sid...","MP3,On-board computer","Alloy wheels,Voice Control","ABS,Central door lock,Daytime running lights,D...",Fahrzeug-Nummer: AM-95365 Ehem. UPE 24.640 EUR...,4 (Green),0,0
3,https://www.autoscout24.com//offers/audi-a1-1-...,Audi A1,1.4 TDi Design S tronic,Sedans,14500,None,"73,000 km",08/2016,1 previous owner,66 kW,",Used,,Diesel (Particulate Filter)",1,0,0,0,99 g CO2/km (comb),Euro 6,Brown,Metallic,0,0,Sedans,3,4,0,Automatic,1422 cc,3,1195 kg,0,Diesel (Particulate Filter),"['3.8 l/100 km (comb)'],['4.3 l/100 km (city)'...",99 g CO2/km (comb),Euro 6,"Air suspension,Armrest,Auxiliary heating,Elect...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport seats,Voice Control","ABS,Alarm system,Central door lock with remote...",Audi A1: - 1e eigenaar - Perfecte staat: schad...,0,6,0
4,https://www.autoscout24.com//offers/audi-a1-sp...,Audi A1,Sportback 1.4 TDI S-Tronic S-Line Ext. admired...,Sedans,16790,None,"16,200 km",05/2016,1 previous owner,66 kW,",Used,,Diesel (Particulate Filter)",1,0,Yes109 g CO2/km (comb),Euro 6,4 (Green),,Black,Metallic,Mythosschwarz Metallic,Cloth Black,Sedans,5,5,0588/BDF,Automatic,1422 cc,3,0,front,Diesel (Particulate Filter),"['4.1 l/100 km (comb)'],['4.6 l/100 km (city)'...",109 g CO2/km (comb),Euro 6,"Air conditioning,Armrest,Automatic climate con...","Bluetooth,CD player,Hands-free equipment,MP3,O...","Alloy wheels,Sport package,Sport suspension,Vo...","ABS,Central door lock,Driver-side airbag,Elect...",Technik & Sicherheit:Xenon plusKlimaautomatikE...,0,0,Germany


In [46]:
pattern = [" ", "[]", "Nan", None, "None", "NaN"]

In [ ]:
# replace np.NaN

In [ ]:
#sc[sc.isnull().any(axis=1)]

In [ ]:
# business.categories=business.categories.apply(lambda row: re.split(", | & ",row))

In [58]:
# jeopardy_data["Float_Value"]=jeopardy_data["Value"].apply(lambda x:0.0 if pd.isna(x) else x.replace('$',''))
